In [20]:
import boto3
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 200)

In [ ]:
## Importing libraries
from random import sample
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.impute import KNNImputer
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'evan-callaghan-bucket'
bucket = s3.Bucket(bucket_name)

file_key = 'Kaggle-American-Express-Default/amex_sample_submission.csv'
file_key2 = 'Kaggle-American-Express-Default/amex_train_labels.csv'
file_key3 = 'Kaggle-American-Express-Default/amex_train_data.csv'
#file_key4 = 'Kaggle-American-Express-Default/amex_test_data.csv'

bucket_object = bucket.Object(file_key)
bucket_object2 = bucket.Object(file_key2)
bucket_object3 = bucket.Object(file_key3)
#bucket_object4 = bucket.Object(file_key4)

file_object = bucket_object.get()
file_object2 = bucket_object2.get()
file_object3 = bucket_object3.get()
#file_object4 = bucket_object4.get()

file_content_stream = file_object.get('Body')
file_content_stream2 = file_object2.get('Body')
file_content_stream3 = file_object3.get('Body')
#file_content_stream4 = file_object4.get('Body')

## Creating data-type dictionary for reading the train data-frame
dtype_dict = {'customer_ID': "object", 'S_2': "object", 'P_2': 'float16', 'D_39': 'float16', 'B_1': 'float16','B_2': 'float16',
              'R_1': 'float16','S_3': 'float16','D_41': 'float16','B_3': 'float16','D_42': 'float16','D_43': 'float16','D_44': 'float16',
              'B_4': 'float16','D_45': 'float16','B_5': 'float16','R_2': 'float16','D_46': 'float16','D_47': 'float16','D_48': 'float16',
              'D_49': 'float16','B_6': 'float16','B_7': 'float16','B_8': 'float16','D_50': 'float16','D_51': 'float16','B_9': 'float16',
              'R_3': 'float16','D_52': 'float16','P_3': 'float16','B_10': 'float16','D_53': 'float16','S_5': 'float16','B_11': 'float16',
              'S_6': 'float16','D_54': 'float16','R_4': 'float16','S_7': 'float16','B_12': 'float16','S_8': 'float16','D_55': 'float16',
              'D_56': 'float16','B_13': 'float16','R_5': 'float16','D_58': 'float16','S_9': 'float16','B_14': 'float16','D_59': 'float16',
              'D_60': 'float16','D_61': 'float16','B_15': 'float16','S_11': 'float16','D_62': 'float16','D_63': 'object','D_64': 'object',
              'D_65': 'float16','B_16': 'float16','B_17': 'float16','B_18': 'float16','B_19': 'float16','D_66': 'float16','B_20': 'float16',
              'D_68': 'float16','S_12': 'float16','R_6': 'float16','S_13': 'float16','B_21': 'float16','D_69': 'float16','B_22': 'float16',
              'D_70': 'float16','D_71': 'float16','D_72': 'float16','S_15': 'float16','B_23': 'float16','D_73': 'float16','P_4': 'float16',
              'D_74': 'float16','D_75': 'float16','D_76': 'float16','B_24': 'float16','R_7': 'float16','D_77': 'float16','B_25': 'float16',
              'B_26': 'float16','D_78': 'float16','D_79': 'float16','R_8': 'float16','R_9': 'float16','S_16': 'float16','D_80': 'float16',
              'R_10': 'float16','R_11': 'float16','B_27': 'float16','D_81': 'float16','D_82': 'float16','S_17': 'float16','R_12': 'float16',
              'B_28': 'float16','R_13': 'float16','D_83': 'float16','R_14': 'float16','R_15': 'float16','D_84': 'float16','R_16': 'float16',
              'B_29': 'float16','B_30': 'float16','S_18': 'float16','D_86': 'float16','D_87': 'float16','R_17': 'float16','R_18': 'float16',
              'D_88': 'float16','B_31': 'int64','S_19': 'float16','R_19': 'float16','B_32': 'float16','S_20': 'float16','R_20': 'float16',
              'R_21': 'float16','B_33': 'float16','D_89': 'float16','R_22': 'float16','R_23': 'float16','D_91': 'float16','D_92': 'float16',
              'D_93': 'float16','D_94': 'float16','R_24': 'float16','R_25': 'float16','D_96': 'float16','S_22': 'float16','S_23': 'float16',
              'S_24': 'float16','S_25': 'float16','S_26': 'float16','D_102': 'float16','D_103': 'float16','D_104': 'float16','D_105': 'float16',
              'D_106': 'float16','D_107': 'float16','B_36': 'float16','B_37': 'float16', 'R_26': 'float16','R_27': 'float16','B_38': 'float16',
              'D_108': 'float16','D_109': 'float16','D_110': 'float16','D_111': 'float16','B_39': 'float16','D_112': 'float16','B_40': 'float16',
              'S_27': 'float16','D_113': 'float16','D_114': 'float16','D_115': 'float16','D_116': 'float16','D_117': 'float16','D_118': 'float16',
              'D_119': 'float16','D_120': 'float16','D_121': 'float16','D_122': 'float16','D_123': 'float16','D_124': 'float16','D_125': 'float16',
              'D_126': 'float16','D_127': 'float16','D_128': 'float16','D_129': 'float16','B_41': 'float16','B_42': 'float16','D_130': 'float16',
              'D_131': 'float16','D_132': 'float16','D_133': 'float16','R_28': 'float16','D_134': 'float16','D_135': 'float16','D_136': 'float16',
              'D_137': 'float16','D_138': 'float16','D_139': 'float16','D_140': 'float16','D_141': 'float16','D_142': 'float16','D_143': 'float16',
              'D_144': 'float16','D_145': 'float16'}

## Reading the data
sample_submission = pd.read_csv(file_content_stream)
train_labels = pd.read_csv(file_content_stream2)
train = pd.read_csv(file_content_stream3, dtype = dtype_dict)
#test = pd.read_csv(file_content_stream4, dtype = dtype_dict)

In [4]:
train = train.merge(train_labels, on = 'customer_ID', how = 'left')

In [5]:
train.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938477,0.001734,0.008728,1.006836,0.009224,0.124023,0.008774,0.004707,...,NaN,NaN,0.002426,0.003706,0.003819,NaN,0.000569,0.000610,0.002674,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936523,0.005775,0.004925,1.000977,0.006153,0.126709,0.000798,0.002714,...,NaN,NaN,0.003956,0.003166,0.005032,NaN,0.009575,0.005493,0.009216,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954102,0.091492,0.021652,1.009766,0.006817,0.123962,0.007599,0.009422,...,NaN,NaN,0.003269,0.007328,0.000427,NaN,0.003429,0.006985,0.002604,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960449,0.002455,0.013687,1.002930,0.001372,0.117188,0.000685,0.005531,...,NaN,NaN,0.006119,0.004517,0.003201,NaN,0.008423,0.006527,0.009598,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947266,0.002483,0.015190,1.000977,0.007607,0.117310,0.004654,0.009308,...,NaN,NaN,0.003672,0.004944,0.008888,NaN,0.001670,0.008125,0.009827,0


In [6]:
train.shape

(5531451, 191)

In [29]:
train['target'].value_counts() / train.shape[0]

0    0.750903
1    0.249097
Name: target, dtype: float64

In [14]:
new_train = train[['customer_ID', 'P_2', 'P_3', 'P_4', 'S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_17', 
               'S_18', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27', 'target']]

In [15]:
new_train['customer_ID'].nunique()

458913

In [30]:
pd.DataFrame(new_train.isnull().sum()).T

,customer_ID,P_2,P_3,P_4,S_3,S_5,S_6,S_7,S_8,S_9,S_11,S_12,S_13,S_15,S_16,S_17,S_18,S_19,S_20,S_22,S_23,S_24,S_25,S_26,S_27,target
0,0,45985,301492,0,1020544,0,0,1020544,0,2933643,0,0,0,0,0,0,0,0,0,19024,445,18593,12847,634,1400935,0


In [34]:
missing = new_train.groupby('target').count().reset_index(drop = False)
missing.head()

,target,customer_ID,P_2,P_3,P_4,S_3,S_5,S_6,S_7,S_8,S_9,S_11,S_12,S_13,S_15,S_16,S_17,S_18,S_19,S_20,S_22,S_23,S_24,S_25,S_26,S_27
0,0,4153582,4119373,3985920,4153582,3241614,4153582,4153582,3241614,4153582,2017455,4153582,4153582,4153582,4153582,4153582,4153582,4153582,4153582,4153582,4137069,4153370,4137438,4149182,4153032,2990399
1,1,1377869,1366093,1244039,1377869,1269293,1377869,1377869,1269293,1377869,580353,1377869,1377869,1377869,1377869,1377869,1377869,1377869,1377869,1377869,1375358,1377636,1375420,1369422,1377785,1140117


In [35]:
new_miss = missing.drop(columns = ['target', 'customer_ID'])

new_miss.iloc[0, :] = (4153582 - new_miss.iloc[0, :]) / 4153582
new_miss.iloc[1, :] = (1377869 - new_miss.iloc[1, :]) / 1377869

new_miss

,P_2,P_3,P_4,S_3,S_5,S_6,S_7,S_8,S_9,S_11,S_12,S_13,S_15,S_16,S_17,S_18,S_19,S_20,S_22,S_23,S_24,S_25,S_26,S_27
0,0.008236,0.040366,0,0.219562,0,0,0.219562,0,0.514286,0,0,0,0,0,0,0,0,0,0.003976,0.000051,0.003887,0.001059,0.000132,0.280043
1,0.008547,0.097128,0,0.078800,0,0,0.078800,0,0.578804,0,0,0,0,0,0,0,0,0,0.001822,0.000169,0.001777,0.006130,0.000061,0.172551


In [31]:
new_miss.iloc[0, :] = new_miss.iloc[0, :] / 3

In [32]:
new_miss

,P_2,P_3,P_4,S_3,S_5,S_6,S_7,S_8,S_9,S_11,S_12,S_13,S_15,S_16,S_17,S_18,S_19,S_20,S_22,S_23,S_24,S_25,S_26,S_27
0,11403,55887.333333,0,303989.333333,0,0,303989.333333,0,712042.333333,0,0,0,0,0,0,0,0,0,5504.333333,70.666667,5381.333333,1466.666667,183.333333,387727.666667
1,11776,133830.000000,0,108576.000000,0,0,108576.000000,0,797516.000000,0,0,0,0,0,0,0,0,0,2511.000000,233.000000,2449.000000,8447.000000,84.000000,237752.000000


#### Subset of data

In [ ]:
## Builidng a subset of the training data-frame with 1000 randomly sampled customers
listy = set(train['customer_ID'].unique())
samples = sample(listy, 1000)

## Subsetting the data
train_sample = train[np.isin(train['customer_ID'], samples)]

## Printing the shape of the resulting data-frame
train_sample.shape

Ideas:

In [ ]:
# from sklearn.linear_model import LinearRegression
# import matplotlib.pyplot as plt

# ## Creating a subset of the training data
# sub = train_sample[train_sample['customer_ID'] == '04305420dfcc6bae8a5943fde9b19a8beff5456e529a0e6bce15655d9b814329']

# ## Defining the input and target variables
# X = np.asarray(sub['S_2'])
# Y = np.asarray(sub['S_5'])

# ## Cleaning the date variable
# df = pd.DataFrame({'time': X, 'value': Y})
# df.time = pd.to_datetime(df.time)

# ## Fitting a linear regression line
# lm_md = LinearRegression().fit(df.time.values.reshape(-1, 1), df['value'].values.reshape(-1, 1)) 

# ## Extracting the linear model coefficients
# print('Coefficient:', lm_md.coef_)

# ## Predicting on the train set
# y_pred = lm_md.predict(df.time.values.astype(float).reshape(-1, 1))
# df['pred'] = y_pred

# ## Plotting results
# ax = df.plot(x='time', y='value', color='black', style='.', figsize = ([14, 8]))
# df.plot(x='time', y='pred', color='orange', linewidth=3, ax=ax, alpha=0.5)
# ax.set_title('My Title')
# ax.set_xlabel('Date')
# ax.set_ylabel('Value')
# plt.show()

In [ ]:
# train_labels[train_labels['customer_ID'] == '04305420dfcc6bae8a5943fde9b19a8beff5456e529a0e6bce15655d9b814329']

In [ ]:
# ## Defining a list of all customer_IDs
# customers = train_samples['customer_ID'].unique()

# ## Initialzing new column in the train_labels_sample data-frame
# train_labels_sample['Spend_Slope'] = np.nan

# ## Looping through each set of customers:
# for customer in customers:
    
#     ## Subsetting by customer
#     customers_temp = train_sample[train_sample['customer_ID'] == customer]
    
#     ## Spend variable subset
#     spend_temp = customer_temp[['S_2', 'S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9','S_11', 'S_12', 'S_13', 'S_15', 'S_16', 
#                                 'S_17', 'S_18', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27']]
    
#     ## Defining the input and target variables
#     X = spend_temp['S_2']
#     Y = spend_temp.drop(columns = ['S_2'])
    
#     ## Creating the linear regression model
#     lm_md = LinearRegression().fit(X, Y)
    
#     ## Extracting the slope of the line
#     slope = lm_md.coef_
    
#     ## Creating new variable for spend slope
#     train_labels_sample['Spend_Slope'] = np.where(train_labels_sample['customer_ID'] == customer, slope, train_labels_sample['Spend_Slope'])
    
    

## Idea:
Create subsets on customer_ID and create a simple linear regression model to extract the slope of the line

Useful for change in spending habit, change in balance, and change in paymne behaviours.

## Handling missing values:

In [ ]:
train.shape

In [ ]:
missing = pd.DataFrame(train.isnull().sum()).T
missing

In [ ]:
train_sample.shape

In [ ]:
missing = pd.DataFrame(train_sample.isnull().sum()).T
missing

In [ ]:
## Dropping columns with too many missing values (over 4,000,000 missing)
train = train.drop(columns = ['D_42', 'D_49', 'D_53', 'D_66', 'D_73', 'D_76', 'R_9', 'D_82', 'B_29', 'D_87', 'D_88', 'D_106', 'R_26', 'D_108', 'D_110', 
                             'D_111', 'B_39', 'B_42', 'D_132', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_142'])
train_sample = train_sample.drop(columns = ['D_42', 'D_49', 'D_53', 'D_66', 'D_73', 'D_76', 'R_9', 'D_82', 'B_29', 'D_87', 'D_88', 'D_106', 'R_26', 
                                            'D_108', 'D_110', 'D_111', 'B_39', 'B_42', 'D_132', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_142'])

### KNN Imputer

#### Sample data-frame:

In [ ]:
## Defining the input variables and dropping categorical variables
knn_train_sample = train_sample.drop(columns = ['customer_ID', 'S_2', 'D_63', 'D_64', 'target'])

## Building the KNN model
knn_train_sample_fixed = KNNImputer(n_neighbors = 5).fit_transform(knn_train_sample)

## Defining data-frames to store results
DF1 = train_sample[['customer_ID', 'S_2', 'D_63', 'D_64', 'target']].reset_index(drop = True)
DF2 = pd.DataFrame(knn_train_sample_fixed, columns = knn_train_sample.columns)

## Returning final data-frame
results = pd.concat([DF1, DF2], axis = 1)

results.head()

#### Full train data-frame:

In [ ]:
## Defining the input variables and dropping categorical variables
knn_train = train.drop(columns = ['customer_ID', 'S_2', 'D_63', 'D_64', 'target'])

## Building the KNN model
knn_train_fixed = KNNImputer(n_neighbors = 5).fit_transform(knn_train)

## Defining data-frames to store results
DF1 = train[['customer_ID', 'S_2', 'D_63', 'D_64', 'target']].reset_index(drop = True)
DF2 = pd.DataFrame(knn_train_fixed, columns = knn_train.columns)

## Returning final data-frame
pd.concat([DF1, DF2], axis = 1)

## "Payment" and "Spend" variable engineering:

In [ ]:
p_variables = train.columns
[x for x in p_variables if x.startswith('P_')]

In [ ]:
## Creating a new figure
fig, axes = plt.subplots(2, 2, figsize = (18, 12))

sns.boxplot(ax = axes[0, 0], x = 'target', y = 'P_2', hue = 'target', data = train)
sns.boxplot(ax = axes[0, 1], x = 'target', y = 'P_3', hue = 'target', data = train)
sns.boxplot(ax = axes[1, 0], x = 'target', y = 'P_4', hue = 'target', data = train)

In [ ]:
## Sum of 'P' variables
sum_value = train.groupby('customer_ID')[['P_2', 'P_3', 'P_4']].sum().reset_index(drop = False)
target = train.groupby('customer_ID')['target'].max().reset_index(drop = False)
target_sums = sum_value.merge(target, how = 'left', on = 'customer_ID')
target_sums.columns = ['customer_ID', 'P_2_sum', 'P_3_sum', 'P_4_sum', 'target']

## Creating a new figure
fig, axes = plt.subplots(2, 2, figsize = (18, 12))

sns.boxplot(ax = axes[0, 0], x = 'target', y = 'P_2_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[0, 1], x = 'target', y = 'P_3_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[1, 0], x = 'target', y = 'P_4_sum', hue = 'target', data = target_sums)

### Selections: P_2, P_2_sum, P_3, P_3_sum

In [ ]:
## Grouping the data by customer_ID to obtain the mean, median, and sum P_2 variable values
P_2_mean = pd.DataFrame(train.groupby(['customer_ID'])['P_2'].mean()).reset_index(drop = False)
P_2_median = pd.DataFrame(train.groupby(['customer_ID'])['P_2'].median()).reset_index(drop = False)
P_2_sum = pd.DataFrame(train.groupby(['customer_ID'])['P_2'].sum()).reset_index(drop = False)

## Cleaning the resulting data-frames
P_2_mean.columns = ['customer_ID', 'P_2_mean']
P_2_median.columns = ['customer_ID', 'P_2_median']
P_2_sum.columns = ['customer_ID', 'P_2_sum']

## Merging the resulting data-frames together
P_2_variable = P_2_mean.merge(P_2_median, how = 'left', 
                              on = 'customer_ID').merge(P_2_sum, how = 'left', on = 'customer_ID')

## Merging back into the complete train data-frame
train = train.merge(P_2_variable, how = 'left', on = 'customer_ID')

In [ ]:
## Grouping the data by customer_ID to obtain the mean, median, and sum P_3 variable values
P_3_mean = pd.DataFrame(train.groupby(['customer_ID'])['P_3'].mean()).reset_index(drop = False)
P_3_median = pd.DataFrame(train.groupby(['customer_ID'])['P_3'].median()).reset_index(drop = False)
P_3_sum = pd.DataFrame(train.groupby(['customer_ID'])['P_3'].sum()).reset_index(drop = False)

## Cleaning the resulting data-frames
P_3_mean.columns = ['customer_ID', 'P_3_mean']
P_3_median.columns = ['customer_ID', 'P_3_median']
P_3_sum.columns = ['customer_ID', 'P_3_sum']

## Merging the resulting data-frames together
P_3_variable = P_3_mean.merge(P_3_median, how = 'left', 
                              on = 'customer_ID').merge(P_3_sum, how = 'left', on = 'customer_ID')

## Merging back into the complete train data-frame
train = train.merge(P_3_variable, how = 'left', on = 'customer_ID')

In [ ]:
train.head(13)

In [ ]:
s_variables = train.columns
[x for x in s_variables if x.startswith('S_')]

In [ ]:
## Creating a new figure
fig, axes = plt.subplots(11, 2, figsize = (18, 100))

sns.boxplot(ax = axes[0, 0], x = 'target', y = 'S_3', hue = 'target', data = train)
sns.boxplot(ax = axes[0, 1], x = 'target', y = 'S_5', hue = 'target', data = train)
sns.boxplot(ax = axes[1, 0], x = 'target', y = 'S_6', hue = 'target', data = train)
sns.boxplot(ax = axes[1, 1], x = 'target', y = 'S_7', hue = 'target', data = train)
sns.boxplot(ax = axes[2, 0], x = 'target', y = 'S_8', hue = 'target', data = train)
sns.boxplot(ax = axes[2, 1], x = 'target', y = 'S_9', hue = 'target', data = train)
sns.boxplot(ax = axes[3, 0], x = 'target', y = 'S_11', hue = 'target', data = train)
sns.boxplot(ax = axes[3, 1], x = 'target', y = 'S_12', hue = 'target', data = train)
sns.boxplot(ax = axes[4, 0], x = 'target', y = 'S_13', hue = 'target', data = train)
sns.boxplot(ax = axes[4, 1], x = 'target', y = 'S_15', hue = 'target', data = train)
sns.boxplot(ax = axes[5, 0], x = 'target', y = 'S_16', hue = 'target', data = train)
sns.boxplot(ax = axes[5, 1], x = 'target', y = 'S_17', hue = 'target', data = train)
sns.boxplot(ax = axes[6, 0], x = 'target', y = 'S_18', hue = 'target', data = train)
sns.boxplot(ax = axes[6, 1], x = 'target', y = 'S_19', hue = 'target', data = train)
sns.boxplot(ax = axes[7, 0], x = 'target', y = 'S_20', hue = 'target', data = train)
sns.boxplot(ax = axes[7, 1], x = 'target', y = 'S_22', hue = 'target', data = train)
sns.boxplot(ax = axes[8, 0], x = 'target', y = 'S_23', hue = 'target', data = train)
sns.boxplot(ax = axes[8, 1], x = 'target', y = 'S_24', hue = 'target', data = train)
sns.boxplot(ax = axes[9, 0], x = 'target', y = 'S_25', hue = 'target', data = train)
sns.boxplot(ax = axes[9, 1], x = 'target', y = 'S_26', hue = 'target', data = train)
sns.boxplot(ax = axes[10, 0], x = 'target', y = 'S_27', hue = 'target', data = train)

In [ ]:
## Sum of 'S' variables
sum_value = train.groupby('customer_ID')[['S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9', 
                                          'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_17', 
                                          'S_18', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 
                                          'S_25', 'S_26', 'S_27']].sum().reset_index(drop = False)
target = train.groupby('customer_ID')['target'].max().reset_index(drop = False)
target_sums = sum_value.merge(target, how = 'left', on = 'customer_ID')
target_sums.columns = ['customer_ID', 'S_3_sum', 'S_5_sum', 'S_6_sum', 'S_7_sum', 'S_8_sum', 'S_9_sum', 
                                          'S_11_sum', 'S_12_sum', 'S_13_sum', 'S_15_sum', 'S_16_sum', 'S_17_sum', 
                                          'S_18_sum', 'S_19_sum', 'S_20_sum', 'S_22_sum', 'S_23_sum', 'S_24_sum', 
                                          'S_25_sum', 'S_26_sum', 'S_27_sum', 'target']

In [ ]:
## Creating a new figure
fig, axes = plt.subplots(11, 2, figsize = (18, 100))

sns.boxplot(ax = axes[0, 0], x = 'target', y = 'S_3_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[0, 1], x = 'target', y = 'S_5_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[1, 0], x = 'target', y = 'S_6_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[1, 1], x = 'target', y = 'S_7_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[2, 0], x = 'target', y = 'S_8_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[2, 1], x = 'target', y = 'S_9_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[3, 0], x = 'target', y = 'S_11_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[3, 1], x = 'target', y = 'S_12_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[4, 0], x = 'target', y = 'S_13_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[4, 1], x = 'target', y = 'S_15_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[5, 0], x = 'target', y = 'S_16_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[5, 1], x = 'target', y = 'S_17_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[6, 0], x = 'target', y = 'S_18_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[6, 1], x = 'target', y = 'S_19_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[7, 0], x = 'target', y = 'S_20_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[7, 1], x = 'target', y = 'S_22_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[8, 0], x = 'target', y = 'S_23_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[8, 1], x = 'target', y = 'S_24_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[9, 0], x = 'target', y = 'S_25_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[9, 1], x = 'target', y = 'S_26_sum', hue = 'target', data = target_sums)
sns.boxplot(ax = axes[10, 0], x = 'target', y = 'S_27_sum', hue = 'target', data = target_sums)

### Selections: S_3, S_3_sum, S_6_sum, S_7, S_7_sum, S_8, S_8_sum, S_13, S_13_sum, S_15, S_27

In [ ]:
## Grouping the data by customer_ID to obtain the mean, median, and sum S_3 variable values
S_3_mean = pd.DataFrame(train.groupby(['customer_ID'])['S_3'].mean()).reset_index(drop = False)
S_3_median = pd.DataFrame(train.groupby(['customer_ID'])['S_3'].median()).reset_index(drop = False)
S_3_sum = pd.DataFrame(train.groupby(['customer_ID'])['S_3'].sum()).reset_index(drop = False)

## Cleaning the resulting data-frames
S_3_mean.columns = ['customer_ID', 'S_3_mean']
S_3_median.columns = ['customer_ID', 'S_3_median']
S_3_sum.columns = ['customer_ID', 'S_3_sum']

## Merging the resulting data-frames together
S_3_variable = S_3_mean.merge(S_3_median, how = 'left', 
                              on = 'customer_ID').merge(S_3_sum, how = 'left', on = 'customer_ID')

## Merging back into the complete train data-frame
train = train.merge(S_3_variable, how = 'left', on = 'customer_ID')

In [ ]:
## Grouping the data by customer_ID to obtain the sum S_6 variable values
S_6_sum = pd.DataFrame(train.groupby(['customer_ID'])['S_6'].sum()).reset_index(drop = False)

## Cleaning the resulting data-frames
S_6_sum.columns = ['customer_ID', 'S_6_sum']

## Merging back into the complete train data-frame
train = train.merge(S_6_sum, how = 'left', on = 'customer_ID')

In [ ]:
## Grouping the data by customer_ID to obtain the mean, median, and sum S_7 variable values
S_7_mean = pd.DataFrame(train.groupby(['customer_ID'])['S_7'].mean()).reset_index(drop = False)
S_7_median = pd.DataFrame(train.groupby(['customer_ID'])['S_7'].median()).reset_index(drop = False)
S_7_sum = pd.DataFrame(train.groupby(['customer_ID'])['S_7'].sum()).reset_index(drop = False)

## Cleaning the resulting data-frames
S_7_mean.columns = ['customer_ID', 'S_7_mean']
S_7_median.columns = ['customer_ID', 'S_7_median']
S_7_sum.columns = ['customer_ID', 'S_7_sum']

## Merging the resulting data-frames together
S_7_variable = S_7_mean.merge(S_7_median, how = 'left', 
                              on = 'customer_ID').merge(S_7_sum, how = 'left', on = 'customer_ID')

## Merging back into the complete train data-frame
train = train.merge(S_7_variable, how = 'left', on = 'customer_ID')

In [ ]:
## Grouping the data by customer_ID to obtain the mean, median, and sum S_8 variable values
S_8_mean = pd.DataFrame(train.groupby(['customer_ID'])['S_8'].mean()).reset_index(drop = False)
S_8_median = pd.DataFrame(train.groupby(['customer_ID'])['S_8'].median()).reset_index(drop = False)
S_8_sum = pd.DataFrame(train.groupby(['customer_ID'])['S_8'].sum()).reset_index(drop = False)

## Cleaning the resulting data-frames
S_8_mean.columns = ['customer_ID', 'S_8_mean']
S_8_median.columns = ['customer_ID', 'S_8_median']
S_8_sum.columns = ['customer_ID', 'S_8_sum']

## Merging the resulting data-frames together
S_8_variable = S_8_mean.merge(S_8_median, how = 'left', 
                              on = 'customer_ID').merge(S_8_sum, how = 'left', on = 'customer_ID')

## Merging back into the complete train data-frame
train = train.merge(S_8_variable, how = 'left', on = 'customer_ID')

In [ ]:
## Grouping the data by customer_ID to obtain the mean, median, and sum S_13 variable values
S_13_mean = pd.DataFrame(train.groupby(['customer_ID'])['S_13'].mean()).reset_index(drop = False)
S_13_median = pd.DataFrame(train.groupby(['customer_ID'])['S_13'].median()).reset_index(drop = False)
S_13_sum = pd.DataFrame(train.groupby(['customer_ID'])['S_13'].sum()).reset_index(drop = False)

## Cleaning the resulting data-frames
S_13_mean.columns = ['customer_ID', 'S_13_mean']
S_13_median.columns = ['customer_ID', 'S_13_median']
S_13_sum.columns = ['customer_ID', 'S_13_sum']

## Merging the resulting data-frames together
S_13_variable = S_13_mean.merge(S_13_median, how = 'left', 
                              on = 'customer_ID').merge(S_13_sum, how = 'left', on = 'customer_ID')

## Merging back into the complete train data-frame
train = train.merge(S_13_variable, how = 'left', on = 'customer_ID')

In [ ]:
## Grouping the data by customer_ID to obtain the mean and median S_15 variable values
S_15_mean = pd.DataFrame(train.groupby(['customer_ID'])['S_15'].mean()).reset_index(drop = False)
S_15_median = pd.DataFrame(train.groupby(['customer_ID'])['S_15'].median()).reset_index(drop = False)

## Cleaning the resulting data-frames
S_15_mean.columns = ['customer_ID', 'S_15_mean']
S_15_median.columns = ['customer_ID', 'S_15_median']

## Merging the resulting data-frames together
S_15_variable = S_15_mean.merge(S_15_median, how = 'left', on = 'customer_ID')

## Merging back into the complete train data-frame
train = train.merge(S_15_variable, how = 'left', on = 'customer_ID')

In [ ]:
## Grouping the data by customer_ID to obtain the mean and median S_27 variable values
S_27_mean = pd.DataFrame(train.groupby(['customer_ID'])['S_27'].mean()).reset_index(drop = False)
S_27_median = pd.DataFrame(train.groupby(['customer_ID'])['S_27'].median()).reset_index(drop = False)

## Cleaning the resulting data-frames
S_27_mean.columns = ['customer_ID', 'S_27_mean']
S_27_median.columns = ['customer_ID', 'S_27_median']

## Merging the resulting data-frames together
S_27_variable = S_27_mean.merge(S_27_median, how = 'left', on = 'customer_ID')

## Merging back into the complete train data-frame
train = train.merge(S_27_variable, how = 'left', on = 'customer_ID')

In [ ]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
## Using the train data-frame to engineer variable interactions

## Defining the input and target variables
X = train[['P_2', 'P_3', 'P_4', 'S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9', 'S_11', 'S_12', 'S_13', 'S_15', 
           'S_16', 'S_17', 'S_18', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27']]
Y = train['target']

## Defining a list to store results
results = []

## Repeating process 100 times
for i in tqdm(range(0, 5)):
    
    ## Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.8, stratify = Y)
    
    ## Building the model
    rf_md = RandomForestClassifier(max_depth = 3, n_estimators = 100).fit(X_train, Y_train)
    
    ## Extracting feature importance scores
    results.append(rf_md.feature_importances_)
    
## Changing results list to a dataframe
results = pd.DataFrame(results, columns = X.columns)

## Computing averages and sorting variables by importance
results = pd.DataFrame(results.apply(np.mean, axis = 0))
results = pd.DataFrame({'Feature': results.index, 'Importance': results[0].values}).sort_values(by = 'Importance', ascending = False)

results